In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn import ensemble,metrics,cross_validation,preprocessing,grid_search
path='C:/Projects/Hackerearth_2/'
from catboost import CatBoost, CatBoostClassifier
import lightgbm as lgb

C:\Users\sharm_000\Anaconda2\envs\py36\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data=pd.read_csv(path+'train.csv',dtype={'roadId':object})
labels=pd.read_csv(path+'labels.csv',dtype={'roadId':object})
test_data=pd.read_csv(path+'test.csv',dtype={'roadId':object})
print (data.shape,labels.shape)
len(np.unique(data['roadId'])),len(np.unique(test_data['roadId']))

(28914, 14) (4825, 3)


(4825, 400)

In [3]:
new_data=pd.merge(data,labels,on='roadId',how='left')
new_data.columns=[i.strip(' ')for i in new_data.columns]
new_data['roadCoordinates_count']=new_data['roadCoordinates'].apply(lambda x: len(x.split()))
new_data['laneLineCoordinates_count']=new_data['laneLineCoordinates'].apply(lambda x: len(x.split()))
new_data=new_data.fillna(0)
new_data.head(2)

,roadId,totalLaneLines,laneLineId,laneLineCoordinates,totalLinesOnLeft,totalLaneLinesOnRight,distFromLaneLineOnLeft,distFromLaneLineOnRight,laneLineLength,roadLength,noOfIntersectingLaneLinesLeft,noOfIntersectingLaneLinesRight,isIntersectingWithRoadGeometry,roadCategory,roadCoordinates,noOfLanes,roadCoordinates_count,laneLineCoordinates_count
0,3177248677,3,X3177248677,11.117636626741222 49.43799310961545|11.11761...,2,0,3.863433276,,4.396337,40.57071,0,0,false,2,11.11747 49.43763|11.11753 49.43776|11.11761 ...,2,5,3
1,3177248677,3,13177248677,11.11771009862968 49.43794191822495|11.117697...,0,2,,3.740901458,37.316844,40.57071,0,0,false,2,11.11747 49.43763|11.11753 49.43776|11.11761 ...,2,5,6


In [4]:
def num_con(x):
    try:
        return float(x)
    except:
        return 0
    
def geom(x):
    if x=='false':
        return 0
    else:
        return 1

In [5]:
new_data['distFromLaneLineOnLeft']=new_data['distFromLaneLineOnLeft'].apply(lambda x: num_con(x))
new_data['distFromLaneLineOnRight']=new_data['distFromLaneLineOnRight'].apply(lambda x: num_con(x))
new_data['isIntersectingWithRoadGeometry']=new_data['isIntersectingWithRoadGeometry'].apply(lambda x: num_con(x))

In [6]:
new_data.columns

Index(['roadId', 'totalLaneLines', 'laneLineId', 'laneLineCoordinates',
       'totalLinesOnLeft', 'totalLaneLinesOnRight', 'distFromLaneLineOnLeft',
       'distFromLaneLineOnRight', 'laneLineLength', 'roadLength',
       'noOfIntersectingLaneLinesLeft', 'noOfIntersectingLaneLinesRight',
       'isIntersectingWithRoadGeometry', 'roadCategory', 'roadCoordinates',
       'noOfLanes', 'roadCoordinates_count', 'laneLineCoordinates_count'],
      dtype='object')

In [7]:
train_pro_data=new_data.groupby(by='roadId').agg({'laneLineId':'nunique','totalLaneLines':'mean','distFromLaneLineOnRight':'mean',\
                                   'distFromLaneLineOnLeft':'mean','laneLineLength':'mean',\
                                  'roadLength':'mean','noOfIntersectingLaneLinesLeft':'mean',\
                                  'roadCategory':'mean','isIntersectingWithRoadGeometry':'mean',\
                                  'laneLineCoordinates_count':'mean','noOfLanes':'mean'}).reset_index()

In [8]:
def laneLineLength_cut(x):
    if x <25:
        return 'A'
    elif x <50:
        return 'B'
    elif x <200:
        return 'C'
    elif x <250:
        return 'D'
    else:
        return 'E'

In [9]:
train_pro_data['roadLength_bucket']=train_pro_data['roadLength'].apply(lambda x: laneLineLength_cut(x))

In [10]:
lbl2=preprocessing.LabelEncoder()
train_pro_data['roadLength_bucket']=lbl2.fit_transform(train_pro_data['roadLength_bucket'])

In [11]:
train_pro_data['noOfLanes']=train_pro_data['noOfLanes'].apply(lambda x:str(x))

In [12]:
lbl=preprocessing.LabelEncoder()
lbl.fit(train_pro_data['noOfLanes'])

LabelEncoder()

In [13]:
train_pro_data['noOfLanes']=lbl.transform(train_pro_data['noOfLanes'])

In [14]:
train_pro_data.shape

(4825, 13)

In [15]:
# new_test_data=pd.merge(test_data,labels,on='roadId',how='left')
test_data.columns=[i.strip(' ')for i in test_data.columns]
test_data.shape,len(np.unique(test_data['roadId']))

((2493, 14), 400)

In [16]:
test_data.head()

,roadId,totalLaneLines,laneLineId,laneLineCoordinates,totalLinesOnLeft,totalLaneLinesOnRight,distFromLaneLineOnLeft,distFromLaneLineOnRight,laneLineLength,roadLength,noOfIntersectingLaneLinesLeft,noOfIntersectingLaneLinesRight,isIntersectingWithRoadGeometry,roadCategory
0,291531296,5,X291531296,11.329899604343693 43.72956824243773|11.33004...,2,2,5.655398294,3.49899823,12.054597,12.056004,0,0,false,1
1,291531296,5,1291531296,11.329900151795446 43.729503478051974|11.3300...,4,0,3.685959755,,12.060141,12.056004,0,0,false,1
2,291531296,5,2291531296,11.329899913660833 43.72953696742431|11.33004...,3,1,3.49899823,3.685959755,12.052468,12.056004,0,0,false,1
3,291531296,5,3291531296,11.329899175813209 43.72961906004278|11.33004...,1,3,3.481580911,5.655398294,12.046642,12.056004,0,0,false,1
4,291531296,5,4291531296,11.329899073264464 43.729650480155186|11.3300...,0,4,,3.481580911,12.054345,12.056004,0,0,false,1


In [17]:
# new_test_data['roadCoordinates_count']=new_test_data['roadCoordinates'].apply(lambda x: len(x.split()))
test_data['laneLineCoordinates_count']=test_data['laneLineCoordinates'].apply(lambda x: len(x.split()))
test_data=test_data.fillna(0)
print (test_data.shape)
test_data.head(2)

(2493, 15)


,roadId,totalLaneLines,laneLineId,laneLineCoordinates,totalLinesOnLeft,totalLaneLinesOnRight,distFromLaneLineOnLeft,distFromLaneLineOnRight,laneLineLength,roadLength,noOfIntersectingLaneLinesLeft,noOfIntersectingLaneLinesRight,isIntersectingWithRoadGeometry,roadCategory,laneLineCoordinates_count
0,291531296,5,X291531296,11.329899604343693 43.72956824243773|11.33004...,2,2,5.655398294,3.49899823,12.054597,12.056004,0,0,false,1,3
1,291531296,5,1291531296,11.329900151795446 43.729503478051974|11.3300...,4,0,3.685959755,,12.060141,12.056004,0,0,false,1,3


In [18]:
test_data['distFromLaneLineOnLeft']=test_data['distFromLaneLineOnLeft'].apply(lambda x: num_con(x))
test_data['distFromLaneLineOnRight']=test_data['distFromLaneLineOnRight'].apply(lambda x: num_con(x))
test_data['isIntersectingWithRoadGeometry']=test_data['isIntersectingWithRoadGeometry'].apply(lambda x: num_con(x))

In [19]:
test_pro_data=test_data.groupby(by='roadId').agg({'laneLineId':'nunique','totalLaneLines':'mean','distFromLaneLineOnRight':'mean',\
                                   'distFromLaneLineOnLeft':'mean','laneLineLength':'mean',\
                                  'roadLength':'mean','noOfIntersectingLaneLinesLeft':'sum',\
                                  'roadCategory':'mean',\
                                  'laneLineCoordinates_count':'mean'}).reset_index()

In [20]:
test_pro_data.shape

(400, 10)

In [21]:
test_pro_data['roadLength_bucket']=test_pro_data['roadLength'].apply(lambda x: laneLineLength_cut(x))
test_pro_data['roadLength_bucket']=lbl2.transform(test_pro_data['roadLength_bucket'])

In [22]:
test_pro_data.columns

Index(['roadId', 'laneLineId', 'totalLaneLines', 'distFromLaneLineOnRight',
       'distFromLaneLineOnLeft', 'laneLineLength', 'roadLength',
       'noOfIntersectingLaneLinesLeft', 'roadCategory',
       'laneLineCoordinates_count', 'roadLength_bucket'],
      dtype='object')

In [23]:
train_pro_data.columns

Index(['roadId', 'laneLineId', 'totalLaneLines', 'distFromLaneLineOnRight',
       'distFromLaneLineOnLeft', 'laneLineLength', 'roadLength',
       'noOfIntersectingLaneLinesLeft', 'roadCategory',
       'isIntersectingWithRoadGeometry', 'laneLineCoordinates_count',
       'noOfLanes', 'roadLength_bucket'],
      dtype='object')

In [24]:
X=test_pro_data.columns.tolist()[1:]
Y='noOfLanes'
len(X)

10

### XGB .773

In [25]:
dtrain=xgb.DMatrix(train_pro_data[X],label=train_pro_data[Y])
dtest=xgb.DMatrix(test_pro_data[X])

In [26]:
def score_f1(preds,dtrain):
#     print ('>>>>',type(preds),preds.shape)
    labels=dtrain.get_label()
    return 'score_f1',metrics.f1_score(labels,preds,average='weighted')

In [27]:
param={}
param['eta']=.1
param['objective']='multi:softmax'
param['num_class']=6
param['max_depth']= 12
param['n_estimators']: 350

In [28]:
xcv=xgb.cv(param,dtrain,num_boost_round=200,feval=score_f1,maximize=True,early_stopping_rounds=50,\
       show_stdv=False,verbose_eval=10)

[0]	train-score_f1:0.795749	test-score_f1:0.52096
[10]	train-score_f1:0.917758	test-score_f1:0.541514
[20]	train-score_f1:0.952763	test-score_f1:0.544338
[30]	train-score_f1:0.967794	test-score_f1:0.544372
[40]	train-score_f1:0.977609	test-score_f1:0.545955
[50]	train-score_f1:0.983231	test-score_f1:0.54685
[60]	train-score_f1:0.98852	test-score_f1:0.546881
[70]	train-score_f1:0.99192	test-score_f1:0.548769
[80]	train-score_f1:0.994816	test-score_f1:0.545748
[90]	train-score_f1:0.996267	test-score_f1:0.545857
[100]	train-score_f1:0.997509	test-score_f1:0.543424
[110]	train-score_f1:0.998544	test-score_f1:0.543908
[120]	train-score_f1:0.999378	test-score_f1:0.540999


In [29]:
num_rounds=len(xcv)
num_rounds

72

In [30]:
watchlist=[(dtrain,'dtrain')]
model_xgb=xgb.train(param,dtrain,evals=watchlist,num_boost_round=num_rounds,feval=score_f1,maximize=True,\
                    early_stopping_rounds=50,\
       verbose_eval=10)

[0]	dtrain-score_f1:0.780648
Will train until dtrain-score_f1 hasn't improved in 50 rounds.
[10]	dtrain-score_f1:0.89915
[20]	dtrain-score_f1:0.935495
[30]	dtrain-score_f1:0.95603
[40]	dtrain-score_f1:0.967975
[50]	dtrain-score_f1:0.974739
[60]	dtrain-score_f1:0.980981
[70]	dtrain-score_f1:0.983455


In [31]:
model_pred=model_xgb.predict(dtest)
model_pred_train=model_xgb.predict(dtest)
np.unique(model_pred,return_counts=True)

(array([ 0.,  1.,  2.,  3.,  4.,  5.], dtype=float32),
 array([ 30, 214, 104,  47,   3,   2], dtype=int64))

In [32]:
model_pred=lbl.inverse_transform(model_pred.astype(int))
np.unique(model_pred)

array(['1', '2', '3', '4', '5', '6'], dtype=object)

In [33]:
sub_samp1=pd.DataFrame(test_pro_data['roadId'])
sub_samp1['noOfLanes']=model_pred

In [34]:
sub_samp1.to_csv(path+'2xgb_model_less_feature.csv',index=False)

In [35]:
train_X,test_X,train_y,test_y=cross_validation.train_test_split(train_pro_data[X],train_pro_data[Y],test_size=.3,random_state =123)
train_X.shape,train_y.shape,test_X.shape,test_y.shape

((3377, 10), (3377,), (1448, 10), (1448,))

### Catboost

In [36]:
clf = CatBoostClassifier(learning_rate=0.01, iterations=1000, random_seed=0, \
                          loss_function='MultiClass', class_count=6)
clf.fit(train_X, train_y,eval_set=(test_X,test_y),use_best_model=True)

In [37]:
metrics.f1_score(test_y,np.ravel(clf.predict(test_X)),average='weighted')

0.54433440217152729

In [38]:
# clf.fit(train_pro_data[X], train_pro_data[Y])
pred_cat=np.ravel(clf.predict(test_pro_data[X]))
pred_cat_train=np.ravel(clf.predict(test_pro_data[X]))
np.unique(pred_cat,return_counts=True)

(array([ 0.,  1.,  2.,  3.,  4.]),
 array([ 22, 214, 134,  29,   1], dtype=int64))

In [39]:
pred_cat=lbl.inverse_transform(pred_cat.astype(int))
print(np.unique(pred_cat))

sub_samp2=pd.DataFrame(test_pro_data['roadId'])
sub_samp2['noOfLanes']=pred_cat

sub_samp2.to_csv(path+'2cat_model_less_feature.csv',index=False)

['1' '2' '3' '4' '5']


### lightgbm

In [40]:
t4_params = {
    'boosting_type': 'gbdt', 'objective': 'multiclass', 'nthread': -1, 'silent': True,
    'num_leaves': 2**4, 'learning_rate': 0.05, 'max_depth': 4,
    'max_bin': 255, 'subsample_for_bin': 50000,
    'subsample': 0.8, 'subsample_freq': 1, 'colsample_bytree': 0.6, 'reg_alpha': 1, 'reg_lambda': 0,
    'min_split_gain': 0.5, 'min_child_weight': 1, 'min_child_samples': 10, 'scale_pos_weight': 1}

# they can be used directly to build a LGBMClassifier (which is wrapped in a sklearn fashion)

In [41]:
ligh_mdl=lgb.LGBMClassifier(**t4_params)

In [42]:
ligh_mdl.fit(train_X, train_y)

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.6, learning_rate=0.05,
        max_bin=255, max_depth=4, min_child_samples=10, min_child_weight=1,
        min_split_gain=0.5, n_estimators=10, nthread=-1, num_leaves=16,
        objective='multiclass', reg_alpha=1, reg_lambda=0,
        scale_pos_weight=1, seed=0, silent=True, subsample=0.8,
        subsample_for_bin=50000, subsample_freq=1)

In [43]:
metrics.f1_score(test_y,ligh_mdl.predict(test_X),average='weighted')

0.50021928463249676

In [44]:
ligh_mdl.fit(train_pro_data[X], train_pro_data[Y])
pred_lgb=ligh_mdl.predict(test_pro_data[X])
pred_lgb_train=ligh_mdl.predict(test_pro_data[X])
np.unique(pred_lgb,return_counts=True)

(array([0, 1, 2, 3, 4], dtype=int64),
 array([ 13, 197, 164,  22,   4], dtype=int64))

In [45]:
pred_lgb=lbl.inverse_transform(pred_lgb.astype(int))
print(np.unique(pred_lgb))

sub_samp3=pd.DataFrame(test_pro_data['roadId'])
sub_samp3['noOfLanes']=pred_lgb

sub_samp3.to_csv(path+'2lgb_model_less_feature.csv',index=False)

['1' '2' '3' '4' '5']


In [46]:
xgb_train_res=(model_xgb.predict(dtrain).astype(int))
cat_train=np.ravel(clf.predict(train_pro_data[X]))
lgb_train=ligh_mdl.predict(train_pro_data[X])
xgb_train_res.shape
new_train_data=pd.DataFrame(data={'xgb':xgb_train_res,'cat':cat_train,'lgb':lgb_train,'noOfLanes':train_pro_data[Y]})

new_train_data.head()

,cat,lgb,noOfLanes,xgb
0,2.0,2,1,1
1,1.0,1,1,1
2,2.0,2,1,1
3,2.0,2,2,2
4,2.0,2,2,2


In [47]:
clf2 = CatBoostClassifier(learning_rate=0.01, iterations=1000, random_seed=0, \
                          loss_function='MultiClass', class_count=6)
clf2.fit(new_train_data[['cat','lgb','xgb']], new_train_data['noOfLanes'])

In [48]:
train_lol=pd.DataFrame(data={'roadId':test_pro_data['roadId'],'xgb':model_pred_train,'cat':np.ravel(pred_cat_train),'lgb':pred_lgb_train})

In [54]:
new_pred=np.ravel(clf2.predict(train_lol[['cat','lgb','xgb']]))
new_pred=lbl.inverse_transform(new_pred.astype(int))

In [55]:
sub_samp4=pd.DataFrame(test_pro_data['roadId'])
sub_samp4['noOfLanes']=new_pred

sub_samp4.to_csv(path+'wierd_feature.csv',index=False)

In [56]:
def mode_data(x,y,z):
    if x==y:
        return x
    elif y==z:
        return y
    else:
        return 'lala'

In [57]:
lolwa=pd.DataFrame(data={'roadId':test_pro_data['roadId'],'xgb':model_pred,'cat':pred_cat,'lgb':pred_lgb})

In [58]:
lolwa['ense']=lolwa[['xgb','cat','lgb']].apply(lambda x: mode_data(*x),axis=1)

In [59]:
def val_repl(x,y):
    if x=='lala':
        return y
    else:
        return x

In [60]:
lolwa['ense']=lolwa[['ense','xgb']].apply(lambda x: val_repl(*x),axis=1)

In [61]:
lolwa=lolwa[['roadId','ense']]
lolwa.columns=['roadId','noOfLanes']

In [62]:
np.unique(lolwa['noOfLanes'],return_counts=True)

(array(['1', '2', '3', '4', '5', '6'], dtype=object),
 array([ 19, 212, 139,  26,   2,   2], dtype=int64))

In [63]:
lolwa.to_csv('2ense_xcl.csv',index=False)